In [1]:
import os
import pennylane as qml
from pennylane import numpy as np
import matplotlib.pyplot as plt
import scipy
from pennylane import BasisState, FermionicSingleExcitation, FermionicDoubleExcitation
import warnings
warnings.filterwarnings("ignore", category=UserWarning)


def inite(elec, orb):
    list1 = []
    # Single excitations
    list1.append([e for e in range(elec)])
    for x in range(elec):
        config = []
        count = orb - elec
        while count < orb:
            for e in range(elec):
                if x == e:
                    config.append(count + (1 if x % 2 else 0))
                else:
                    config.append(e)
            list1.append(config)
            config = []
            count += 2

    # Double excitations
    for x in range(elec):
        for y in range(x + 1, elec):
            for count1 in range(orb - elec, orb, 2):
                for count2 in range(orb - elec, orb, 2):
                    cont = 1 if (count1 != count2 or (x % 2) != (y % 2)) else 0
                    if (x % 2) == (y % 2) and count2 < count1:
                        cont = 0
                    if cont == 1:
                        config = []
                        for e in range(elec):
                            if x == e:
                                config.append(count1 + (1 if x % 2 else 0))
                            elif y == e:
                                config.append(count2 + (1 if y % 2 else 0))
                            else:
                                config.append(e)
                        list1.append(config)
    return list1



In [2]:
def apply_readout_error(p_ro, wires):
    """Apply a symmetric bit-flip channel before measurement on each wire."""
    for w in wires:
        qml.BitFlip(p_ro, wires=w)

def _nontrivial_wire_count(op):
    """
    Count how many qubits a Pauli term acts non-trivially on.
    Works across PennyLane versions.
    """
    # Tensor products have .obs (list of factors)
    if hasattr(op, "obs"):
        factors = op.obs
    else:
        factors = [op]

    count = 0
    for f in factors:
        if f.name != "Identity":
            count += len(f.wires)
    return count


def mitigate_pauli_expval(expval_meas, op, p_ro):
    """
    Mitigate symmetric readout bit-flip for a Pauli-string expectation value.

    For a Pauli string acting on k qubits:
      <P>_meas = (1 - 2 p_ro)^k * <P>_true
      => <P>_mit = <P>_meas / (1 - 2 p_ro)^k
    """
    k = _nontrivial_wire_count(op)
    if k == 0:
        return expval_meas  # constant/identity term

    scale = (1.0 - 2.0 * p_ro) ** k
    if abs(scale) < 1e-10:
        return expval_meas  # avoid blow-up if p_ro ~ 0.5

    expval_mit = expval_meas / scale
    # optional clip (helps with finite-shot noise)
    return np.clip(expval_mit, -1.0, 1.0)

def mitigated_energy_from_terms(term_expvals, H, p_ro):
    """Combine term-by-term expvals into mitigated energy."""
    E = 0.0
    for val, c, op in zip(term_expvals, H.coeffs, H.ops):
        E = E + c * mitigate_pauli_expval(val, op, 0.02)
    return E

In [3]:
def is_global_identity(op):
    """True if op is qml.Identity with no wires."""
    return (op.name == "Identity") and (len(op.wires) == 0)

def split_hamiltonian_terms(H):
    """
    Returns:
      const_shift: sum of coefficients for global Identity terms
      coeffs_ops: list of (coeff, op) for measurable terms
    """
    const_shift = 0.0
    coeffs_ops = []
    for c, op in zip(H.coeffs, H.ops):
        if is_global_identity(op):
            const_shift += c
        else:
            coeffs_ops.append((c, op))
    return const_shift, coeffs_ops

def mitigated_energy_from_terms(term_expvals, coeffs_ops, const_shift, p_ro):
    E = const_shift
    for val, (c, op) in zip(term_expvals, coeffs_ops):
        E = E + c * mitigate_pauli_expval(val, op, p_ro)
    return E


In [12]:
def gs_exact(symbols, geometry, electrons, charge, shots=None, max_iter=100):
    # Molecular Hamiltonian
    H, qubits = qml.qchem.molecular_hamiltonian(symbols, geometry, basis="sto-3g", charge=charge, method="pyscf")

    const_shift, coeffs_ops = split_hamiltonian_terms(H)
    ops_meas = [op for _, op in coeffs_ops]

    # Hartree-Fock reference
    hf_state = qml.qchem.hf_state(electrons, qubits)

    # Single and double excitations
    singles, doubles = qml.qchem.excitations(electrons, qubits)
    s_wires, d_wires = qml.qchem.excitations_to_wires(singles, doubles)
    pool = singles + doubles
    wires = list(range(qubits))
    params = np.zeros(len(pool))
   
    # Device
    if shots==0:
        dev = qml.device("default.mixed", wires=qubits)
    else:
        dev = qml.device("default.mixed", wires=qubits,shots=shots)

    
    @qml.qnode(dev, interface="autograd")
    def circuit(params, wires, s_wires, d_wires, hf_state):
        # Initialize HF state
        qml.BasisState(hf_state, wires=wires)
        # Ensure params shape compatible
        param_array = params
        if len(params.shape) == 1:
            param_array = np.expand_dims(params, 0)

        # Apply UCCSD excitations
        for layer in range(1):
            # Double excitations
            for i, (w1, w2) in enumerate(d_wires):
                qml.FermionicDoubleExcitation(param_array[layer][len(s_wires)+i], wires1=w1, wires2=w2)
            # Single excitations
            for j, s_w in enumerate(s_wires):
                qml.FermionicSingleExcitation(param_array[layer][j], wires=s_w)
            for w in wires:
                qml.BitFlip(p=0.02, wires=w)
        return qml.expval(H)
        #return [qml.expval(op) for op in ops_meas]
    optimizer = qml.GradientDescentOptimizer(stepsize=2.0)
    for n in range(max_iter):
        params, energy = optimizer.step_and_cost(
            circuit, params,
            wires=wires, s_wires=s_wires,
            d_wires=d_wires, hf_state=hf_state
        )

    #def cost_fn(p):
        #term_vals = circuit(p, wires, s_wires, d_wires, hf_state)
        #return mitigated_energy_from_terms(term_vals, coeffs_ops, const_shift, 0.02)
    
    #optimizer = qml.GradientDescentOptimizer(stepsize=2.0)
    #for n in range(max_iter):
    #    params, energy = optimizer.step_and_cost(
     #       cost_fn, params
     #   )

    #term_vals = circuit(params, wires, s_wires, d_wires, hf_state)
   #E_mit = mitigated_energy_from_terms(term_vals, coeffs_ops, const_shift, 0.02)

    gr_state = circuit(params, wires, s_wires, d_wires, hf_state)
    print(f"Ground state energy: {gr_state:.8f} Ha")
    print(f"Mitigated ground state energy: {E_mit:.8f} Ha")
    return params, pool

def qsceom(symbols, geometry, active_electrons,  active_orbitals, charge,params,shots=0):
    # Build the electronic Hamiltonian
    H, qubits = qml.qchem.molecular_hamiltonian(symbols, geometry, basis="sto-3g", method="pyscf", active_electrons=active_electrons, active_orbitals=active_orbitals, charge=charge)
    const_shift, coeffs_ops = split_hamiltonian_terms(H)
    ops_meas = [op for _, op in coeffs_ops]
    singles, doubles = qml.qchem.excitations(active_electrons, qubits)
    # Map excitations to the wires the UCCSD circuit will act on
    s_wires, d_wires = qml.qchem.excitations_to_wires(singles, doubles)
    wires=range(qubits)
    #print(qubits)
    #print(wires)
    #num_qubits = len(qubits)
    null_state = np.zeros(qubits,int)
    list1 = inite(active_electrons,qubits)
    values =[]
  
    #for t in range(1):
    if shots==0:
        dev = qml.device("default.mixed", wires=qubits)
    else:
        dev = qml.device("default.mixed", wires=qubits,shots=shots)
    #circuit for diagonal part
   
    @qml.qnode(dev)
    def circuit_d(params, occ, wires, s_wires, d_wires, hf_state):
        BasisState(hf_state, wires=wires)
        for w in occ:
            qml.PauliX(wires=w)
        param_array = params
        if len(params.shape) == 1:
            param_array = np.expand_dims(params, 0)
       
        for i, (w1, w2) in enumerate(d_wires):
            FermionicDoubleExcitation(param_array[0][len(s_wires) + i], wires1=w1, wires2=w2)
        for j, s_w in enumerate(s_wires):
            FermionicSingleExcitation(param_array[0][j], wires=s_w)
        for w in wires:
            qml.BitFlip(p=0.02, wires=w)
        return qml.expval(H)
        #return [qml.expval(op) for op in ops_meas]

    #circuit for off-diagonal part
    @qml.qnode(dev)
    def circuit_od(params, occ1, occ2, wires, s_wires, d_wires, hf_state):
        BasisState(hf_state, wires=wires)
        for w in occ1:
            qml.PauliX(wires=w)
        first = -1
        for v in occ2:
            if v not in occ1:
                if first == -1:
                    first = v
                    qml.Hadamard(wires=v)
                else:
                    qml.CNOT(wires=[first, v])
        for v in occ1:
            if v not in occ2:
                if first == -1:
                    first = v
                    qml.Hadamard(wires=v)
                else:
                    qml.CNOT(wires=[first, v])
       
        param_array = params
        if len(params.shape) == 1:
            param_array = np.expand_dims(params, 0)
        
        for i, (w1, w2) in enumerate(d_wires):
            FermionicDoubleExcitation(param_array[0][len(s_wires) + i], wires1=w1, wires2=w2)
        for j, s_w in enumerate(s_wires):
            FermionicSingleExcitation(param_array[0][j], wires=s_w)
        for w in wires:
            qml.BitFlip(p=0.02, wires=w)
        return qml.expval(H)
        #return [qml.expval(op) for op in ops_meas]
    
    #def mitigated_H_expect_from_terms(term_vals):
        #return mitigated_energy_from_terms(term_vals, coeffs_ops, const_shift, 0.02)

    #final M matrix
    M = np.zeros((len(list1),len(list1)))
    for i in range(len(list1)):
        for j in range(len(list1)):
            if i == j:
                M[i,i]= circuit_d(params, list1[i], wires, s_wires, d_wires, null_state)
                #Mtmp = circuit_d(params, list1[i], wires, s_wires, d_wires, null_state)
                #M[i,i] = mitigated_H_expect_from_terms(Mtmp)
    for i in range(len(list1)):
        for j in range(len(list1)):
            if i!=j:
                Mtmp = circuit_od(params, list1[i],list1[j],wires, s_wires, d_wires, null_state)
                #mit = mitigated_H_expect_from_terms(Mtmp)
                M[i,j]= Mtmp-M[i,i]/2.0-M[j,j]/2.0
    eig,evec=np.linalg.eig(M)
    values.append(np.sort(eig))
    return values

r = 1.88973
#symbols = ['H', 'H', 'H', 'H']
symbols = ['H', 'H']
geometry = np.array([
    [0.0, 0.0, 0.0],
    [0.0, 0.0, 0.735*r]
    #[0.0, 1.5*r, 0.0],
    #[0.0, 1.5*r, 1.5*r]
], requires_grad=False)

electrons = 2
charge = 0

active_electrons = 2
active_orbitals = 2

params, pool = gs_exact(symbols, geometry, electrons, charge, max_iter=200)
eig = qsceom(symbols, geometry, active_electrons, active_orbitals, charge, params)
#eigval.append(eig)
print("p=0.02 Bit-Flip Noise Model")
print('eigenvalues:', eig)

Ground state energy: -1.08507327 Ha


NameError: name 'E_mit' is not defined

In [13]:
import pennylane as qml
import numpy as np
 
n_qubits = 8  # whatever your active space gives you
dev = qml.device("default.mixed", wires=n_qubits, shots=20_000)
 
def parity_operator(wires):
    P = qml.PauliZ(wires[0])
    for w in wires[1:]:
        P = P @ qml.PauliZ(w)
    return P
 
def apply_leakage_noise(wires, p_leak=0.01, gamma=0.0):
    """Noise that breaks particle-number/parity."""
    for w in wires:
        # occupation-changing stochastic error
        qml.BitFlip(p_leak, wires=w)
        # optional amplitude damping (also breaks N)
        if gamma > 0:
            qml.AmplitudeDamping(gamma, wires=w)
 
# ---- Replace this with your UCCSD block ----
def uccsd_block(theta, wires):
    # placeholder: put your UCCSD template here
    # e.g., qml.templates.UCCSD(theta, wires=wires, ... )
    # or your own compiled UCCSD circuit
    pass
# -------------------------------------------
 
@qml.qnode(dev)
def noisy_uccsd_expectations(theta, H_terms, p_leak=0.01, gamma=0.0):
    wires = list(range(n_qubits))
 
    # 1) Prepare HF state (placeholder; use your hf_state)
    # qml.BasisState(hf_state, wires=wires)
 
    # 2) UCCSD ansatz
    uccsd_block(theta, wires=wires)
 
    # 3) Inject “bad” noise that violates parity/number
    apply_leakage_noise(wires, p_leak=p_leak, gamma=gamma)
 
    # 4) Build measurement list:
    P = parity_operator(wires)
 
    # Measure <P>
    measurements = [qml.expval(P)]
 
    # For energy: H = sum_k c_k O_k
    # Measure <O_k> and <P O_k> so you can compute <H> and <PH>.
    for c_k, O_k in H_terms:
        measurements.append(qml.expval(O_k))
        measurements.append(qml.expval(P @ O_k))
 
    return tuple(measurements)
 
def projected_energy_from_samples(result, H_terms):
    """
    result layout:
      result[0] = <P>
      then for each term k:
<O_k>, <P O_k>
    """
    Pexp = result[0]
    idx = 1
    Hexp = 0.0
    PHexp = 0.0
    for (c_k, O_k) in H_terms:
        Ok = result[idx]; POk = result[idx+1]
        idx += 2
        Hexp += c_k * Ok
        PHexp += c_k * POk
 
    E_proj = (Hexp + PHexp) / (1.0 + Pexp)   # parity + sector
    return Hexp, E_proj, Pexp

In [ ]:
def gs_exact(symbols, geometry, electrons, charge, shots=None, max_iter=100):
    # Molecular Hamiltonian
    H, qubits = qml.qchem.molecular_hamiltonian(symbols, geometry, basis="sto-3g", charge=charge, method="pyscf")

    # Hartree-Fock reference
    hf_state = qml.qchem.hf_state(electrons, qubits)

    # Single and double excitations
    singles, doubles = qml.qchem.excitations(electrons, qubits)
    s_wires, d_wires = qml.qchem.excitations_to_wires(singles, doubles)
    pool = singles + doubles
    wires = list(range(qubits))
    params = np.zeros(len(pool))
    # Device
    if shots==0:
        dev = qml.device("lightning.qubit", wires=qubits)
    else:
        dev = qml.device("lightning.qubit", wires=qubits,shots=shots)

    
    @qml.qnode(dev, interface="autograd")
    def circuit(params, wires, s_wires, d_wires, hf_state):
        # Initialize HF state
        BasisState(hf_state, wires=wires)
        # Ensure params shape compatible
        param_array = params
        if len(params.shape) == 1:
            param_array = np.expand_dims(params, 0)

        # Apply UCCSD excitations
        for layer in range(1):
            # Double excitations
            for i, (w1, w2) in enumerate(d_wires):
                FermionicDoubleExcitation(param_array[layer][len(s_wires)+i], wires1=w1, wires2=w2)
            # Single excitations
            for j, s_w in enumerate(s_wires):
                FermionicSingleExcitation(param_array[layer][j], wires=s_w)
        return qml.expval(H)
    
    optimizer = qml.GradientDescentOptimizer(stepsize=2.0)
    for n in range(max_iter):
        params, energy = optimizer.step_and_cost(
            circuit, params,
            wires=wires, s_wires=s_wires,
            d_wires=d_wires, hf_state=hf_state
        )

    gr_state = circuit(params, wires, s_wires, d_wires, hf_state)
    print(f"Ground state energy: {gr_state:.8f} Ha")
    return params, pool

def qsceom(symbols, geometry, active_electrons,  active_orbitals, charge,params,shots=0):
    # Build the electronic Hamiltonian
    H, qubits = qml.qchem.molecular_hamiltonian(symbols, geometry, basis="sto-3g", method="pyscf", active_electrons=active_electrons, active_orbitals=active_orbitals, charge=charge)
    singles, doubles = qml.qchem.excitations(active_electrons, qubits)
    # Map excitations to the wires the UCCSD circuit will act on
    s_wires, d_wires = qml.qchem.excitations_to_wires(singles, doubles)
    wires=range(qubits)
    #print(qubits)
    #print(wires)
    #num_qubits = len(qubits)
    null_state = np.zeros(qubits,int)
    list1 = inite(active_electrons,qubits)
    values =[]
  
    #for t in range(1):
    if shots==0:
        dev = qml.device("default.mixed", wires=qubits)
    else:
        dev = qml.device("default.mixed", wires=qubits,shots=shots)
    #circuit for diagonal part
   
    @qml.qnode(dev)
    def circuit_d(params, occ, wires, s_wires, d_wires, hf_state):
        BasisState(hf_state, wires=wires)
        for w in occ:
            qml.PauliX(wires=w)
        param_array = params
        if len(params.shape) == 1:
            param_array = np.expand_dims(params, 0)
       
        for i, (w1, w2) in enumerate(d_wires):
            FermionicDoubleExcitation(param_array[0][len(s_wires) + i], wires1=w1, wires2=w2)
        for j, s_w in enumerate(s_wires):
            FermionicSingleExcitation(param_array[0][j], wires=s_w)
        #for w in wires:
            #qml.BitFlip(p=0.04, wires=w)
        return qml.expval(H)

    #circuit for off-diagonal part
    @qml.qnode(dev)
    def circuit_od(params, occ1, occ2, wires, s_wires, d_wires, hf_state):
        BasisState(hf_state, wires=wires)
        for w in occ1:
            qml.PauliX(wires=w)
        first = -1
        for v in occ2:
            if v not in occ1:
                if first == -1:
                    first = v
                    qml.Hadamard(wires=v)
                else:
                    qml.CNOT(wires=[first, v])
        for v in occ1:
            if v not in occ2:
                if first == -1:
                    first = v
                    qml.Hadamard(wires=v)
                else:
                    qml.CNOT(wires=[first, v])
       
        param_array = params
        if len(params.shape) == 1:
            param_array = np.expand_dims(params, 0)
        
        for i, (w1, w2) in enumerate(d_wires):
            FermionicDoubleExcitation(param_array[0][len(s_wires) + i], wires1=w1, wires2=w2)
        for j, s_w in enumerate(s_wires):
            FermionicSingleExcitation(param_array[0][j], wires=s_w)
        #for w in wires:
            #qml.BitFlip(p=0.04, wires=w)
        return qml.expval(H)
    #final M matrix
    M = np.zeros((len(list1),len(list1)))
    for i in range(len(list1)):
        for j in range(len(list1)):
            if i == j:
                M[i,i] = circuit_d(params, list1[i], wires, s_wires, d_wires, null_state)
    for i in range(len(list1)):
        for j in range(len(list1)):
            if i!=j:
                Mtmp = circuit_od(params, list1[i],list1[j],wires, s_wires, d_wires, null_state)
                M[i,j]=Mtmp-M[i,i]/2.0-M[j,j]/2.0
    eig,evec=np.linalg.eig(M)
    values.append(np.sort(eig))
    return values

r = 1.88973
symbols = ['H', 'H', 'H', 'H']
#symbols = ['H', 'H']
geometry = np.array([
    [0.0, 0.0, 0.0],
    [0.0, 0.0, 1.5*r],
    [0.0, 1.5*r, 0.0],
    [0.0, 1.5*r, 1.5*r]
], requires_grad=False)

electrons = 4
charge = 0

active_electrons = 4
active_orbitals = 4
params, pool = gs_exact(symbols, geometry, electrons, charge, max_iter=200)

eig = qsceom(symbols, geometry, active_electrons, active_orbitals, charge, params)
#eigval.append(eig)
print("p=0.04 Bit-Flip Noise Model")
print('eigenvalues:', eig)


In [ ]:
def gs_exact(symbols, geometry, electrons, charge, shots=None, max_iter=100):
    # Molecular Hamiltonian
    H, qubits = qml.qchem.molecular_hamiltonian(symbols, geometry, basis="sto-3g", charge=charge, method="pyscf")

    # Hartree-Fock reference
    hf_state = qml.qchem.hf_state(electrons, qubits)

    # Single and double excitations
    singles, doubles = qml.qchem.excitations(electrons, qubits)
    s_wires, d_wires = qml.qchem.excitations_to_wires(singles, doubles)
    pool = singles + doubles
    wires = list(range(qubits))
    params = np.zeros(len(pool))
    
    # Device
    if shots==0:
        dev = qml.device("default.mixed", wires=qubits)
    else:
        dev = qml.device("default.mixed", wires=qubits,shots=shots)

    
    @qml.qnode(dev, interface="autograd")
    def circuit(params, wires, s_wires, d_wires, hf_state):
        # Initialize HF state
        BasisState(hf_state, wires=wires)
        # Ensure params shape compatible
        param_array = params
        if len(params.shape) == 1:
            param_array = np.expand_dims(params, 0)

        # Apply UCCSD excitations
        for layer in range(1):
            # Double excitations
            for i, (w1, w2) in enumerate(d_wires):
                FermionicDoubleExcitation(param_array[layer][len(s_wires)+i], wires1=w1, wires2=w2)
            # Single excitations
            for j, s_w in enumerate(s_wires):
                FermionicSingleExcitation(param_array[layer][j], wires=s_w)
            for w in wires:
                qml.BitFlip(p=0.06, wires=w)
        return qml.expval(H)
    
    optimizer = qml.GradientDescentOptimizer(stepsize=2.0)
    for n in range(max_iter):
        params, energy = optimizer.step_and_cost(
            circuit, params,
            wires=wires, s_wires=s_wires,
            d_wires=d_wires, hf_state=hf_state
        )

    gr_state = circuit(params, wires, s_wires, d_wires, hf_state)
    print(f"Ground state energy: {gr_state:.8f} Ha")
    return params, pool

def qsceom(symbols, geometry, active_electrons,  active_orbitals, charge,params,shots=0):
    # Build the electronic Hamiltonian
    H, qubits = qml.qchem.molecular_hamiltonian(symbols, geometry, basis="sto-3g", method="pyscf", active_electrons=active_electrons, active_orbitals=active_orbitals, charge=charge)
    singles, doubles = qml.qchem.excitations(active_electrons, qubits)
    # Map excitations to the wires the UCCSD circuit will act on
    s_wires, d_wires = qml.qchem.excitations_to_wires(singles, doubles)
    wires=range(qubits)
    #print(qubits)
    #print(wires)
    #num_qubits = len(qubits)
    null_state = np.zeros(qubits,int)
    list1 = inite(active_electrons,qubits)
    values =[]
  
    #for t in range(1):
    if shots==0:
        dev = qml.device("default.mixed", wires=qubits)
    else:
        dev = qml.device("default.mixed", wires=qubits,shots=shots)
    #circuit for diagonal part
   
    @qml.qnode(dev)
    def circuit_d(params, occ, wires, s_wires, d_wires, hf_state):
        BasisState(hf_state, wires=wires)
        for w in occ:
            qml.PauliX(wires=w)
        param_array = params
        if len(params.shape) == 1:
            param_array = np.expand_dims(params, 0)
       
        for i, (w1, w2) in enumerate(d_wires):
            FermionicDoubleExcitation(param_array[0][len(s_wires) + i], wires1=w1, wires2=w2)
        for j, s_w in enumerate(s_wires):
            FermionicSingleExcitation(param_array[0][j], wires=s_w)
        #for w in wires:
            #qml.BitFlip(p=0.06, wires=w)
        return qml.expval(H)

    #circuit for off-diagonal part
    @qml.qnode(dev)
    def circuit_od(params, occ1, occ2, wires, s_wires, d_wires, hf_state):
        BasisState(hf_state, wires=wires)
        for w in occ1:
            qml.PauliX(wires=w)
        first = -1
        for v in occ2:
            if v not in occ1:
                if first == -1:
                    first = v
                    qml.Hadamard(wires=v)
                else:
                    qml.CNOT(wires=[first, v])
        for v in occ1:
            if v not in occ2:
                if first == -1:
                    first = v
                    qml.Hadamard(wires=v)
                else:
                    qml.CNOT(wires=[first, v])
       
        param_array = params
        if len(params.shape) == 1:
            param_array = np.expand_dims(params, 0)
        
        for i, (w1, w2) in enumerate(d_wires):
            FermionicDoubleExcitation(param_array[0][len(s_wires) + i], wires1=w1, wires2=w2)
        for j, s_w in enumerate(s_wires):
            FermionicSingleExcitation(param_array[0][j], wires=s_w)
        #for w in wires:
            #qml.BitFlip(p=0.06, wires=w)
        return qml.expval(H)
    #final M matrix
    M = np.zeros((len(list1),len(list1)))
    for i in range(len(list1)):
        for j in range(len(list1)):
            if i == j:
                M[i,i] = circuit_d(params, list1[i], wires, s_wires, d_wires, null_state)
    for i in range(len(list1)):
        for j in range(len(list1)):
            if i!=j:
                Mtmp = circuit_od(params, list1[i],list1[j],wires, s_wires, d_wires, null_state)
                M[i,j]=Mtmp-M[i,i]/2.0-M[j,j]/2.0
    eig,evec=np.linalg.eig(M)
    values.append(np.sort(eig))
    return values

r = 1.88973
symbols = ['H', 'H', 'H', 'H']
#symbols = ['H', 'H']
geometry = np.array([
    [0.0, 0.0, 0.0],
    [0.0, 0.0, 1.5*r],
    [0.0, 1.5*r, 0.0],
    [0.0, 1.5*r, 1.5*r]
], requires_grad=False)

electrons = 4
charge = 0

active_electrons = 4
active_orbitals = 4
params, pool = gs_exact(symbols, geometry, electrons, charge, max_iter=200)

eig = qsceom(symbols, geometry, active_electrons, active_orbitals, charge, params)
#eigval.append(eig)
print("p=0.06 Bit-Flip Noise Model")
print('eigenvalues:', eig)


In [ ]:
def gs_exact(symbols, geometry, electrons, charge, shots=None, max_iter=100):
    # Molecular Hamiltonian
    H, qubits = qml.qchem.molecular_hamiltonian(symbols, geometry, basis="sto-3g", charge=charge, method="pyscf")

    # Hartree-Fock reference
    hf_state = qml.qchem.hf_state(electrons, qubits)

    # Single and double excitations
    singles, doubles = qml.qchem.excitations(electrons, qubits)
    s_wires, d_wires = qml.qchem.excitations_to_wires(singles, doubles)
    pool = singles + doubles
    wires = list(range(qubits))
    params = np.zeros(len(pool))
   
    # Device
    if shots==0:
        dev = qml.device("default.mixed", wires=qubits)
    else:
        dev = qml.device("default.mixed", wires=qubits,shots=shots)

    
    @qml.qnode(dev, interface="autograd")
    def circuit(params, wires, s_wires, d_wires, hf_state):
        # Initialize HF state
        BasisState(hf_state, wires=wires)
        # Ensure params shape compatible
        param_array = params
        if len(params.shape) == 1:
            param_array = np.expand_dims(params, 0)

        # Apply UCCSD excitations
        for layer in range(1):
            # Double excitations
            for i, (w1, w2) in enumerate(d_wires):
                FermionicDoubleExcitation(param_array[layer][len(s_wires)+i], wires1=w1, wires2=w2)
            # Single excitations
            for j, s_w in enumerate(s_wires):
                FermionicSingleExcitation(param_array[layer][j], wires=s_w)
            for w in wires:
                qml.BitFlip(p=0.08, wires=w)
        return qml.expval(H)
    
    optimizer = qml.GradientDescentOptimizer(stepsize=2.0)
    for n in range(max_iter):
        params, energy = optimizer.step_and_cost(
            circuit, params,
            wires=wires, s_wires=s_wires,
            d_wires=d_wires, hf_state=hf_state
        )

    gr_state = circuit(params, wires, s_wires, d_wires, hf_state)
    print(f"Ground state energy: {gr_state:.8f} Ha")
    return params, pool

def qsceom(symbols, geometry, active_electrons,  active_orbitals, charge,params,shots=0):
    # Build the electronic Hamiltonian
    H, qubits = qml.qchem.molecular_hamiltonian(symbols, geometry, basis="sto-3g", method="pyscf", active_electrons=active_electrons, active_orbitals=active_orbitals, charge=charge)
    singles, doubles = qml.qchem.excitations(active_electrons, qubits)
    # Map excitations to the wires the UCCSD circuit will act on
    s_wires, d_wires = qml.qchem.excitations_to_wires(singles, doubles)
    wires=range(qubits)
    #print(qubits)
    #print(wires)
    #num_qubits = len(qubits)
    null_state = np.zeros(qubits,int)
    list1 = inite(active_electrons,qubits)
    values =[]
  
    #for t in range(1):
    if shots==0:
        dev = qml.device("default.mixed", wires=qubits)
    else:
        dev = qml.device("default.mixed", wires=qubits,shots=shots)
    #circuit for diagonal part
   
    @qml.qnode(dev)
    def circuit_d(params, occ, wires, s_wires, d_wires, hf_state):
        BasisState(hf_state, wires=wires)
        for w in occ:
            qml.PauliX(wires=w)
        param_array = params
        if len(params.shape) == 1:
            param_array = np.expand_dims(params, 0)
       
        for i, (w1, w2) in enumerate(d_wires):
            FermionicDoubleExcitation(param_array[0][len(s_wires) + i], wires1=w1, wires2=w2)
        for j, s_w in enumerate(s_wires):
            FermionicSingleExcitation(param_array[0][j], wires=s_w)
        #for w in wires:
            #qml.BitFlip(p=0.08, wires=w)
        return qml.expval(H)

    #circuit for off-diagonal part
    @qml.qnode(dev)
    def circuit_od(params, occ1, occ2, wires, s_wires, d_wires, hf_state):
        BasisState(hf_state, wires=wires)
        for w in occ1:
            qml.PauliX(wires=w)
        first = -1
        for v in occ2:
            if v not in occ1:
                if first == -1:
                    first = v
                    qml.Hadamard(wires=v)
                else:
                    qml.CNOT(wires=[first, v])
        for v in occ1:
            if v not in occ2:
                if first == -1:
                    first = v
                    qml.Hadamard(wires=v)
                else:
                    qml.CNOT(wires=[first, v])
       
        param_array = params
        if len(params.shape) == 1:
            param_array = np.expand_dims(params, 0)
        
        for i, (w1, w2) in enumerate(d_wires):
            FermionicDoubleExcitation(param_array[0][len(s_wires) + i], wires1=w1, wires2=w2)
        for j, s_w in enumerate(s_wires):
            FermionicSingleExcitation(param_array[0][j], wires=s_w)
        #for w in wires:
            #qml.BitFlip(p=0.08, wires=w)
        return qml.expval(H)
    #final M matrix
    M = np.zeros((len(list1),len(list1)))
    for i in range(len(list1)):
        for j in range(len(list1)):
            if i == j:
                M[i,i] = circuit_d(params, list1[i], wires, s_wires, d_wires, null_state)
    for i in range(len(list1)):
        for j in range(len(list1)):
            if i!=j:
                Mtmp = circuit_od(params, list1[i],list1[j],wires, s_wires, d_wires, null_state)
                M[i,j]=Mtmp-M[i,i]/2.0-M[j,j]/2.0
    eig,evec=np.linalg.eig(M)
    values.append(np.sort(eig))
    return values

r = 1.88973
symbols = ['H', 'H', 'H', 'H']
#symbols = ['H', 'H']
geometry = np.array([
    [0.0, 0.0, 0.0],
    [0.0, 0.0, 1.5*r],
    [0.0, 1.5*r, 0.0],
    [0.0, 1.5*r, 1.5*r]
], requires_grad=False)

electrons = 4
charge = 0

active_electrons = 4
active_orbitals = 4
params, pool = gs_exact(symbols, geometry, electrons, charge, max_iter=200)

eig = qsceom(symbols, geometry, active_electrons, active_orbitals, charge, params)
#eigval.append(eig)
print("p=0.08 Bit-Flip Noise Model")
print('eigenvalues:', eig)


In [ ]:
def gs_exact(symbols, geometry, electrons, charge, shots=None, max_iter=100):
    # Molecular Hamiltonian
    H, qubits = qml.qchem.molecular_hamiltonian(symbols, geometry, basis="sto-3g", charge=charge, method="pyscf")

    # Hartree-Fock reference
    hf_state = qml.qchem.hf_state(electrons, qubits)

    # Single and double excitations
    singles, doubles = qml.qchem.excitations(electrons, qubits)
    s_wires, d_wires = qml.qchem.excitations_to_wires(singles, doubles)
    pool = singles + doubles
    wires = list(range(qubits))
    params = np.zeros(len(pool))
    
    # Device
    if shots==0:
        dev = qml.device("default.mixed", wires=qubits)
    else:
        dev = qml.device("default.mixed", wires=qubits,shots=shots)

    
    @qml.qnode(dev, interface="autograd")
    def circuit(params, wires, s_wires, d_wires, hf_state):
        # Initialize HF state
        BasisState(hf_state, wires=wires)
        # Ensure params shape compatible
        param_array = params
        if len(params.shape) == 1:
            param_array = np.expand_dims(params, 0)

        # Apply UCCSD excitations
        for layer in range(1):
            # Double excitations
            for i, (w1, w2) in enumerate(d_wires):
                FermionicDoubleExcitation(param_array[layer][len(s_wires)+i], wires1=w1, wires2=w2)
            # Single excitations
            for j, s_w in enumerate(s_wires):
                FermionicSingleExcitation(param_array[layer][j], wires=s_w)
            for w in wires:
                qml.BitFlip(p=0.1, wires=w)
        return qml.expval(H)
    
    optimizer = qml.GradientDescentOptimizer(stepsize=2.0)
    for n in range(max_iter):
        params, energy = optimizer.step_and_cost(
            circuit, params,
            wires=wires, s_wires=s_wires,
            d_wires=d_wires, hf_state=hf_state
        )

    gr_state = circuit(params, wires, s_wires, d_wires, hf_state)
    print(f"Ground state energy: {gr_state:.8f} Ha")
    return params, pool

def qsceom(symbols, geometry, active_electrons,  active_orbitals, charge,params,shots=0):
    # Build the electronic Hamiltonian
    H, qubits = qml.qchem.molecular_hamiltonian(symbols, geometry, basis="sto-3g", method="pyscf", active_electrons=active_electrons, active_orbitals=active_orbitals, charge=charge)
    singles, doubles = qml.qchem.excitations(active_electrons, qubits)
    # Map excitations to the wires the UCCSD circuit will act on
    s_wires, d_wires = qml.qchem.excitations_to_wires(singles, doubles)
    wires=range(qubits)
    #print(qubits)
    #print(wires)
    #num_qubits = len(qubits)
    null_state = np.zeros(qubits,int)
    list1 = inite(active_electrons,qubits)
    values =[]
  
    #for t in range(1):
    if shots==0:
        dev = qml.device("default.mixed", wires=qubits)
    else:
        dev = qml.device("default.mixed", wires=qubits,shots=shots)
    #circuit for diagonal part
   
    @qml.qnode(dev)
    def circuit_d(params, occ, wires, s_wires, d_wires, hf_state):
        BasisState(hf_state, wires=wires)
        for w in occ:
            qml.PauliX(wires=w)
        param_array = params
        if len(params.shape) == 1:
            param_array = np.expand_dims(params, 0)
       
        for i, (w1, w2) in enumerate(d_wires):
            FermionicDoubleExcitation(param_array[0][len(s_wires) + i], wires1=w1, wires2=w2)
        for j, s_w in enumerate(s_wires):
            FermionicSingleExcitation(param_array[0][j], wires=s_w)
        #for w in wires:
            #qml.BitFlip(p=0.1, wires=w)
        return qml.expval(H)

    #circuit for off-diagonal part
    @qml.qnode(dev)
    def circuit_od(params, occ1, occ2, wires, s_wires, d_wires, hf_state):
        BasisState(hf_state, wires=wires)
        for w in occ1:
            qml.PauliX(wires=w)
        first = -1
        for v in occ2:
            if v not in occ1:
                if first == -1:
                    first = v
                    qml.Hadamard(wires=v)
                else:
                    qml.CNOT(wires=[first, v])
        for v in occ1:
            if v not in occ2:
                if first == -1:
                    first = v
                    qml.Hadamard(wires=v)
                else:
                    qml.CNOT(wires=[first, v])
       
        param_array = params
        if len(params.shape) == 1:
            param_array = np.expand_dims(params, 0)
        
        for i, (w1, w2) in enumerate(d_wires):
            FermionicDoubleExcitation(param_array[0][len(s_wires) + i], wires1=w1, wires2=w2)
        for j, s_w in enumerate(s_wires):
            FermionicSingleExcitation(param_array[0][j], wires=s_w)
        #for w in wires:
            #qml.BitFlip(p=0.1, wires=w)
        return qml.expval(H)
    #final M matrix
    M = np.zeros((len(list1),len(list1)))
    for i in range(len(list1)):
        for j in range(len(list1)):
            if i == j:
                M[i,i] = circuit_d(params, list1[i], wires, s_wires, d_wires, null_state)
    for i in range(len(list1)):
        for j in range(len(list1)):
            if i!=j:
                Mtmp = circuit_od(params, list1[i],list1[j],wires, s_wires, d_wires, null_state)
                M[i,j]=Mtmp-M[i,i]/2.0-M[j,j]/2.0
    eig,evec=np.linalg.eig(M)
    values.append(np.sort(eig))
    return values

r = 1.88973
symbols = ['H', 'H', 'H', 'H']
#symbols = ['H', 'H']
geometry = np.array([
    [0.0, 0.0, 0.0],
    [0.0, 0.0, 1.5*r],
    [0.0, 1.5*r, 0.0],
    [0.0, 1.5*r, 1.5*r]
], requires_grad=False)

electrons = 4
charge = 0

active_electrons = 4
active_orbitals = 4
params, pool = gs_exact(symbols, geometry, electrons, charge, max_iter=200)

eig = qsceom(symbols, geometry, active_electrons, active_orbitals, charge, params)
#eigval.append(eig)
print("p=0.1 Bit-Flip Noise Model")
print('eigenvalues:', eig)
